In [1]:
from os import listdir
from os.path import isfile, join, isdir
import sys, getopt
import subprocess

import pandas as pd


In [2]:
PATH_TO_STUDY_FOLDER = 'C:\\Users\\igorp\\University of South Florida\\Mao, Wenbin - Myocardium (organized)\\Active\\Guccione_oneElem_study\\study2'
PATH_TO_PARAMS_FILE = PATH_TO_STUDY_FOLDER + "\\" + 'modified_param_log.csv'
PATH_TO_OUTPUT_FILE = 'C:\\Users\\igorp\\University of South Florida\\Mao, Wenbin - Myocardium (organized)\\Active\\Guccione_oneElem_study\\pickleData\\data.pickle'

In [3]:
def find_files(path_to_folder, condition=None):
	if (condition):
		if (condition[0] == "fileFormat"):
			_l = len(condition[1])
			return [(join(path_to_folder,f), f,f[:-_l - 1]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f)) and f[-_l:] == condition[1]]
		else:
			raise(ValueError("Condition should be a tuple where the first argument is the condition name and the second is its value"))
	else:
		return [(join(path_to_folder,f), f, f.rsplit(".")[0]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]

print("-- Getting files")
files = find_files(PATH_TO_STUDY_FOLDER, ("fileFormat","pickle"))
print("Number of files found:", len(files))

-- Getting files
Number of files found: 7


In [4]:
def compile_data(files):
  frames = []
  for i, (fp, ff, fn) in enumerate(files):
    ndf = pd.read_pickle(fp)
    frames.append(ndf)

  df = pd.concat(frames, ignore_index=True)
  return df

print("-- Compiling data")
df = compile_data(files)

-- Compiling data


In [5]:
print(df.loc[0,:])

a       1.000000e+00
b       1.000000e+00
time    1.000000e-03
fail    0.000000e+00
x_1    -5.000000e-01
y_1    -5.000000e-01
z_1     0.000000e+00
x_2    -5.000000e-01
y_2    -5.000000e-01
z_2     1.000160e+00
x_3    -5.000000e-01
y_3     5.000120e-01
z_3     0.000000e+00
x_4    -5.000000e-01
y_4     5.000120e-01
z_4     1.000160e+00
x_5     4.998300e-01
y_5    -5.000000e-01
z_5     0.000000e+00
x_6     4.998300e-01
y_6    -5.000000e-01
z_6     1.000160e+00
x_7     4.998300e-01
y_7     5.000120e-01
z_7     0.000000e+00
x_8     4.998300e-01
y_8     5.000120e-01
z_8     1.000160e+00
ux_1    0.000000e+00
uy_1    0.000000e+00
uz_1    0.000000e+00
ux_2    0.000000e+00
uy_2    0.000000e+00
uz_2    1.575250e-04
ux_3    0.000000e+00
uy_3    1.243420e-05
uz_3    0.000000e+00
ux_4    0.000000e+00
uy_4    1.243420e-05
uz_4    1.575250e-04
ux_5   -1.699330e-04
uy_5    0.000000e+00
uz_5    0.000000e+00
ux_6   -1.699330e-04
uy_6    0.000000e+00
uz_6    1.575250e-04
ux_7   -1.699330e-04
uy_7    1.243

In [6]:
df.tail()

,a,b,time,fail,x_1,y_1,z_1,x_2,y_2,z_2,...,uz_7,ux_8,uy_8,uz_8,sx,sy,sz,sxy,sxz,syz
68035,3.647059,5.764706,0.185130,0.0,-0.5,-0.5,0.0,-0.5,-0.5,1.06815,...,0.0,-0.068955,0.005536,0.068148,0.000439,0.000489,0.000470,3.124450e-12,-1.012750e-12,1.013210e-12
68036,3.647059,5.764706,0.189527,0.0,-0.5,-0.5,0.0,-0.5,-0.5,1.06928,...,0.0,-0.070048,0.005650,0.069283,-0.000505,-0.000562,-0.000539,-3.084330e-12,1.004490e-12,-1.002950e-12
68037,3.647059,5.764706,0.194227,0.0,-0.5,-0.5,0.0,-0.5,-0.5,1.07051,...,0.0,-0.071239,0.005781,0.070515,0.000396,0.000442,0.000426,2.960620e-12,-9.688910e-13,9.655970e-13
68038,3.647059,5.764706,0.198927,0.0,-0.5,-0.5,0.0,-0.5,-0.5,1.07169,...,0.0,-0.072368,0.005902,0.071689,-0.000482,-0.000538,-0.000517,-2.903580e-12,9.549440e-13,-9.497070e-13
68039,3.647059,5.764706,0.200000,0.0,-0.5,-0.5,0.0,-0.5,-0.5,1.07203,...,0.0,-0.072705,0.005948,0.072028,0.003130,0.003482,0.003347,1.191220e-11,-3.980790e-12,3.952490e-12


In [7]:
print("-- Checking for missing parameters")
import csv

def load_params_vals(file):
	params = {}
	params_2 = set()
	with open(file, 'r',newline='') as csvfile:
		spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
		for i, row in enumerate(spamreader):
			params[i] = [float(v) for v in row[-1].split(";")]
			params_2.add(tuple(params[i]))
	return params, params_2


params, params_2 = load_params_vals(PATH_TO_PARAMS_FILE)

has_failed = False
for index, row in df.iterrows():
	a = row['a']
	b = row['b']
	if (a, b) not in params_2:
		has_failed = True
		print("fail at", [a, b])

assert has_failed == False

print(" No parameters are missing")


-- Checking for missing parameters
 No parameters are missing


In [8]:
print("-- Saving new pickle")
df.to_pickle(PATH_TO_OUTPUT_FILE)

-- Saving new pickle
